In [ ]:
############################################################################################
# IMPORTS
############################################################################################
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#import tensorflow.keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import ConfusionMatrixDisplay


import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
#import tensorflow.keras as keras

from tensorflow.keras import layers
from tensorflow.keras import regularizers


from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Activation, Flatten

from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical



from IPython import display
from PIL import Image


import pathlib
import shutil
import tempfile
import concurrent

import pickle
import visualkeras

import json
import glob





print(tf.__version__)
print(tf.config.list_physical_devices())
############################################################################################

In [ ]:
############################################################################################
# 1. CONSTANTS - PATHS
############################################################################################

#DATA_FS251 = './data/iFood_2019'
DATA_FS251 = './data/food-101'
CLASSES_FILE_NAME = '/formated_annot/classes_formated.csv'

TRAIN_INFO = '/annot/train_info.csv'
VAL_INFO = '/annot/val_info.csv'
TEST_INFO = '/annot/test_info.csv'

TRAIN_PICS_PATH = './data/iFood_2019/train_set/'
TEST_PICS_PATH = './data/iFood_2019/test_set/'
VAL_PICS_PATH = './data/iFood_2019/val_set/'

MODELS = './models/'

SEED = 111

#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
gpu_dev = tf.config.experimental.list_physical_devices('GPU')
for itm in gpu_dev:
    tf.config.experimental.set_memory_growth(itm, True)


############################################################################################
# IMPORTING DATA
############################################################################################
df_classes = pd.read_csv(DATA_FS251 + CLASSES_FILE_NAME)
df_train = pd.read_csv(DATA_FS251 + TRAIN_INFO, names=['file_name', 'class_num'])
df_validate = pd.read_csv(DATA_FS251 + VAL_INFO, names=['file_name', 'class_num'])
df_test = pd.read_csv(DATA_FS251 + TEST_INFO, names=['file_name'])

df_train.head(3)

In [ ]:
############################################################################################
# 2. CONSTANTS - MODEL
############################################################################################
training_history = dict()

#N_TRAIN = len(df_train.iloc[:, 0])
EPOCHS = 30
BATCH_SIZE = 64              # 8 fit into GPU RAM, 64 fit into system RAM
RESIZE_TO = (512, 512)
#STEPS_PER_EPOCH = N_TRAIN // BATCH_SIZE

############################################################################################
# FUNCTIONS, DEFFINITIONS
############################################################################################

class CRelu(tf.keras.Layer):
    def __init__(self, axis=-1, **kwargs):
        self.axis = axis 
        super(CRelu, self).__init__(**kwargs)

    def build(self, input_shape):
        super(CRelu, self).build(input_shape)

    def call(self, x):
        x = tf.nn.crelu(x, axis=self.axis)
        return x

    def compute_output_shape(self, input_shape):
        output_shape = list(input_shape)
        output_shape[-1] = output_shape[-1] * 2
        output_shape = tuple(output_shape)
        return output_shape

    def get_config(self, input_shape):
        config = {'axis': self.axis, }
        base_config = super(CReLU, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
############################################################################################
# import image files
############################################################################################

train_pics, val_pics = tf.keras.preprocessing.image_dataset_from_directory(
    './data/food-101/images/',
    labels='inferred',
#    label_mode='int',
    label_mode='categorical',
    class_names=None,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
#    image_size=RESIZE_TO,
    shuffle=True,
    seed=SEED,
    validation_split=0.2,
    subset='both',
#    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=False,
    pad_to_aspect_ratio=False,
    data_format='channels_last',
    verbose=True
)
#test_pics = tf.keras.preprocessing.image_dataset_from_directory(
#    VAL_PICS_PATH,
#    labels='inferred',
#    label_mode='int',
#    label_mode='categorical',
#    class_names=None,
#    color_mode='rgb',
#    batch_size=BATCH_SIZE,
#    image_size=RESIZE_TO,
#    shuffle=True,
#    seed=SEED,
#    validation_split=None,
#    subset=None,
#    interpolation='bilinear',
#    follow_links=False,
#    crop_to_aspect_ratio=False,
#    pad_to_aspect_ratio=False,
#    data_format='channels_last',
#    verbose=True
#)
#true_test_pics = tf.keras.preprocessing.image_dataset_from_directory(
#    TEST_PICS_PATH,
#    labels=None,
#    label_mode=None,
#    class_names=None,
#    color_mode='rgb',
#    batch_size=BATCH_SIZE,
#    image_size=RESIZE_TO,
#    shuffle=True,
#    seed=SEED,
#    validation_split=None,
#    subset=None,
#    interpolation='bilinear',
#    follow_links=False,
#    crop_to_aspect_ratio=False,
#    pad_to_aspect_ratio=False,
#    data_format='channels_last',
#    verbose=True
#)

In [ ]:
train_pics.class_names
#val_pics.as_numpy_iterator().next()[0]
#val_pics.take(1)

In [ ]:
############################################################################################
# DEFINING THE MODEL
############################################################################################
tf.keras.backend.clear_session()

core_model = tf.keras.applications.EfficientNetB0(include_top=False, input_shape=(240, 240, 3))
core_model.trainable = True
#for layer in core_model.layers[:-5]:
#    layer.trainable = False

inputed = tf.keras.Input(shape=(256,256,3))
x = tf.keras.layers.Resizing(height = 240,width = 240,interpolation='bilinear',crop_to_aspect_ratio=False,pad_to_aspect_ratio=False,fill_mode='constant',fill_value=0.0)(inputed)
x = core_model(x)
x = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.L2(0.01))(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(1024, activation='relu', activity_regularizer=tf.keras.regularizers.L2())(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(256, activation='relu', activity_regularizer=tf.keras.regularizers.L2())(x)
x = tf.keras.layers.BatchNormalization()(x)
outputed = tf.keras.layers.Dense(units=101,activation='softmax',activity_regularizer=tf.keras.regularizers.L2(), name='Output_layer')(x)


model_supclass = tf.keras.Model(inputed,outputed)

model_supclass.summary()


In [ ]:
############################################################################################
# SCHEDULER PARAMS
############################################################################################
#lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
#    0.00005, #0.000005
#    decay_steps=STEPS_PER_EPOCH*100, #100
#    decay_rate=1,
#    staircase=False)

In [ ]:
############################################################################################
# COMPILING
############################################################################################

model_supclass.compile(
        optimizer = tf.keras.optimizers.Nadam(learning_rate=0.05),                  #2
        loss = tf.keras.losses.CategoricalCrossentropy(),
        metrics = [tf.keras.metrics.Accuracy()]
        )
############################################################################################
# FITTING
############################################################################################


with tf.device('/GPU:0'):
    model_history_1 = model_supclass.fit(
        train_pics,
        epochs = 5, 
        batch_size = BATCH_SIZE,
        callbacks = tf.keras.callbacks.EarlyStopping(
                            monitor='val_loss',
                            min_delta=1,
                            patience=3,
                            verbose=1,
                            mode='auto',
                            baseline=None,
                            restore_best_weights=True,
                            start_from_epoch=0),
        validation_data = val_pics
    )

In [ ]:
############################################################################################
# COMPILING 2
############################################################################################

model_supclass.compile(
#        optimizer = tf.keras.optimizers.Adam(learning_rate = lr_schedule),                     
#        optimizer = tf.keras.optimizers.Adam(),                                     #
#        optimizer = tf.keras.optimizers.Adadelta(learning_rate=0.5),                 #3            
#        optimizer = tf.keras.optimizers.Adagrad(),                                             
#        optimizer = tf.keras.optimizers.Adam(0.001),
#        optimizer = tf.keras.optimizers.Nadam(),
#        optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.05),                 #1
        optimizer = tf.keras.optimizers.Nadam(learning_rate=0.005),                  #2
#        loss = tf.keras.losses.CTC(),
#        loss = tf.keras.losses.KLDivergence(),
#        loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.3),
        loss = tf.keras.losses.CategoricalCrossentropy(),
#        loss = tf.keras.losses.BinaryCrossentropy(),
#        loss = tf.keras.losses.CategoricalCrossentropy(),
#        loss = tf.keras.losses.SparseCategoricalCrossentropy(),
#        metrics = [tf.keras.metrics.CategoricalCrossentropy(from_logits=False, label_smoothing=0.2, axis=-1),
#                   tf.keras.metrics.F1Score(average=None, threshold=0.5, dtype=None),
#                   tf.keras.metrics.CategoricalAccuracy()]
        metrics = [tf.keras.metrics.Accuracy()]
#        metrics = [tf.keras.metrics.F1Score(average='micro', threshold=0.5),tf.keras.metrics.CategoricalAccuracy()]
        )
############################################################################################
# FITTING 2
############################################################################################


with tf.device('/GPU:0'):
    model_history_2 = model_supclass.fit(
        train_pics,
        epochs = 20, 
        batch_size = BATCH_SIZE,
        callbacks = tf.keras.callbacks.EarlyStopping(
                            monitor='val_loss',
                            min_delta=0.2,
                            patience=5,
                            verbose=1,
                            mode='auto',
                            baseline=None,
                            restore_best_weights=True,
                            start_from_epoch=0),
        validation_data = val_pics
    )

In [ ]:
############################################################################################
# SAVING MODEL
############################################################################################
MODEL_NAME = 'food-101-EfNetB0-A0.__-earlystopP0.2-E_of_-B64_softCross_v0.6'
os.makedirs( os.path.dirname(MODELS + 'test/' + MODEL_NAME + '/'), exist_ok=True)
model_supclass.save(MODELS + 'test/' + MODEL_NAME + '/'+ MODEL_NAME + ".keras")
#model_supclass_loaded = load_model(MODELS + MODEL_NAME + '/'+ MODEL_NAME + ".keras")
#model_supclass_loaded.summary()

# with pickle
import pickle
with open(MODELS + 'test/' + MODEL_NAME + '/'+ MODEL_NAME + ".pckl", 'wb') as file_pi:
    pickle.dump(model_history.history, file_pi)

# history in csv format
hits_df = []
hits_df = pd.DataFrame(model_history.history)
hits_df.to_csv(MODELS + 'test/' + MODEL_NAME + '/'+ MODEL_NAME + '.hist_csv')


In [ ]:
############################################################################################
# EVALUATION - MODEL STRUCTURE IMAGES
############################################################################################
tf.keras.utils.plot_model(
    model_supclass,
    to_file=MODELS + 'test/' + MODEL_NAME + '/'+ MODEL_NAME + '_model_diagram.png',
    show_shapes=True,
    show_dtype=True,
    show_layer_names=False,
    rankdir='TB',
    expand_nested=False,
    dpi=200,
    show_layer_activations=True,
    show_trainable=True
)

import visualkeras
visualkeras.layered_view(model_supclass,
                        to_file=MODELS + 'test/' + MODEL_NAME + '/'+ MODEL_NAME + '_layer_plot.png',
                        legend=True,
                        max_xy=250,
                        min_z = 1, 
                        max_z= 75,
                        scale_z = 5,
#                        color_map = color_map,
#                        show_nested = True,
                        spacing=50
#                        show_dimension=True
                        )

In [ ]:
############################################################################################
# EVALUATION - ACCURACY
############################################################################################

plt.legend(bbox_to_anchor = [1, 1.02])
plt.plot(model_history.history['accuracy'],label='Train accuracy')
plt.plot(model_history.history['val_accuracy'],label='Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig(MODELS + 'test/' + MODEL_NAME + '/'+ MODEL_NAME + '_ACCURACY.png')
plt.show()

In [ ]:
############################################################################################
# EVALUATION - LOSS
############################################################################################

plt.legend(bbox_to_anchor = [1, 1.02])
plt.plot(model_history.history['loss'],label='Train loss')
plt.plot(model_history.history['val_loss'],label='Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.savefig(MODELS + 'test/' + MODEL_NAME + '/'+ MODEL_NAME + '_LOSS.png')
plt.legend()
plt.show()

# Print out the score
score = model_supclass.evaluate(val_pics , batch_size=1, verbose=1)
print(score, model_supclass.metrics_names)

In [ ]:
############################################################################################
# PREDICTION
############################################################################################

y_test_pred = np.array([])
y_test =  np.array([])
for x, y in val_pics:
        y_test_pred = np.concatenate([y_test_pred, np.argmax(model_supclass.predict(x, verbose=0), axis = -1)])
        y_test = np.concatenate([y_test, np.argmax(y.numpy(), axis=-1)])

In [ ]:
for x, y in val_pics:
    print(y.shape)

In [ ]:
############################################################################################
# PREDICTION - CONFMATRIX
############################################################################################

from sklearn.metrics import ConfusionMatrixDisplay
plt.rcParams['figure.figsize'] = (20, 20)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true= y_test, y_pred=y_test_pred)
#disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=np.unique(y_test))
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.xticks(rotation=90)
plt.savefig(MODELS + 'test/' + MODEL_NAME + '/'+ MODEL_NAME + '_CONFMAT.png')
plt.show()






In [ ]:
plt.figure(figsize=(15, 15))
for images, labels in val_pics.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(str( labels[i].numpy().astype(int) ) + str(np.argmax(model_supclass.predict(images[i], verbose=0), axis = -1))  )
        plt.axis("off")
        

In [ ]:
#plt.imshow(images[0].numpy().astype("uint8"))
#plt.show()
